In [ ]:
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt

import dlmhelper.data
import dlmhelper.tools

In [ ]:
dataset = sm.datasets.co2.load()

In [ ]:
print(sm.datasets.co2.TITLE)

In [ ]:
dataset.data.co2

Now we create a time series object from this data using the following inputs:

- **time64 & data** : provide the data with corresponding time (given as numpy.datetime64) values
- **product_type** : allows to save some identifier with the data
- **time_unit** : the time unit declares what a single time step corresponds to



In [ ]:
time64 = np.asarray(dataset.data.index)
data = np.asarray(dataset.data.co2)

product_type="Mauna_Loa_CO2"
time_unit = "week"

In [ ]:
#Uncomment to see the docstring for TimeSeries
#?dlmhelper.data.TimeSeries

In [ ]:
timeseries = dlmhelper.data.TimeSeries(data, time64 = time64, time_unit=time_unit, product_type=product_type)

The timeseries can be quickly plotted

In [ ]:
_fig, _ax = timeseries.plot()

Next we perform a single dynamic linear fit on the time series

In [ ]:
result = dlmhelper.tools.dlm_fit(timeseries, "Mauna_Loa_CO2_test",level=True, variable_level=False,trend=True, variable_trend=True,
                    seasonal=True, autoregressive=1, irregular=True, seasonal_period = [52], 
                       )

In [ ]:
#Uncomment to see the docstring for dlm_fit
#?dlmhelper.tools.dlm_fit

We can quicky visualize an overview of the results

In [ ]:
fig = result.plot_summary(figsize=(16,8))

It is also possible to perform an ensemble fit on the data using a single command. We keep the standard values for the level and trend component and 

In [ ]:
seasonal = [True, False]

seasonal_period = [
                [52]
                ]
seasonal_harmonics = [
                    [[1,2]]
                    ]

variable_seasonal = [
                    [[False]]
                    ]

autoregressive = [None,1]
irregular = [True, False]

In [ ]:
result_list = dlmhelper.tools.dlm_ensemble(timeseries, "CO2_ensemble_test",
                                           seasonal=seasonal, seasonal_period=seasonal_period,
                                           seasonal_harmonics=seasonal_harmonics,variable_seasonal = variable_seasonal,
                                           autoregressive=autoregressive, irregular=irregular)

In [ ]:
result_list.summary()

In [ ]:
result_list.plot_summary(n=8, figsize=(16,8))

The best fit (according to some chose measure) can be also easily extracted:

In [ ]:
best_fit = result_list.get_best_result(sort='aic')
fig = best_fit.plot_summary(figsize=(12,8))

In [ ]:
gr = []
gr_err = []
years = np.arange(1960,2000,1)
for y in years:
    _gr, _gr_err = dlmhelper.tools.annual_level_increase(best_fit, y, tolerance=np.timedelta64(5,'D'))
    gr.append(_gr)
    gr_err.append(_gr_err)
    
gr = np.array(gr)
gr_err = np.array(gr_err)

In [ ]:
gr

In [ ]:
fig = plt.figure(figsize=(14,14))
ax = fig.add_subplot(111)

ax.bar(years,gr,width=1, edgecolor="black", yerr = gr_err)